In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
from mle_toolbox import load_result_logs, visualize
from neuroevobench.metrics import plot_rliable

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


envpool not installed, Atari problems will not work.
envpool not installed, Atari problems will not work.
You need to install brax for Brax tasks:
  pip install git+https://github.com/google/brax.git@main


### Helper to load and normalize results

In [2]:
result_dirs = [
               "../experiments/fmnist_classify/eval/",
               "../experiments/mnist_generate/eval/",
               "../experiments/ant/eval/",
               "../experiments/fetch/eval/",
               "../experiments/addition/eval/",
               "../experiments/smnist/eval/",
               "../experiments/asterix/eval/",
               "../experiments/breakout/eval/",
               "../experiments/cifar/eval/"
               ]

strategy_names = ['OpenES', 'PGPE', 'ARS', 'SNES', 'Sep_CMA_ES', 'LES',
                  'SimpleGA', 'SAMR_GA', 'GESMR_GA', 'LGA']
color_palette = sns.color_palette("colorblind", n_colors=len(strategy_names))

In [3]:
def load_and_normalize(log_path, normalize_by="OpenES"):
    meta_log, hyper_log = load_result_logs(log_path, aggregate_seeds=True)
    min_perf = hyper_log.test_eval_perf.min()
    max_perf = hyper_log.test_eval_perf.max()
    
    results_dict = {}
    for i, s in enumerate(hyper_log.strategy_name):
        results_dict[s] = hyper_log.test_eval_perf[i]

    for k, v in results_dict.items():
        results_dict[k] = (v - min_perf) / (max_perf - min_perf)
    
    norm_perf = float(results_dict[normalize_by])
    for k, v in results_dict.items():
        results_dict[k] = v / norm_perf

    return results_dict

    
all_dicts = []
for dir in result_dirs:
    d = load_and_normalize(dir, normalize_by="OpenES")
    all_dicts.append(d)

dd = defaultdict(list)

for d in all_dicts:
    for key, value in d.items():
        dd[key].append(value)

score_dict = dict(dd)

for k, v in score_dict.items():
    score_dict[k] = np.array(v).reshape(-1, 1)

del score_dict["OpenES"]

/Users/rob/anaconda3/envs/mle-toolbox/lib/python3.9/site-packages/numpy/lib/function_base.py:4650: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(
/Users/rob/anaconda3/envs/mle-toolbox/lib/python3.9/site-packages/numpy/lib/function_base.py:4488: RuntimeWarning: invalid value encountered in add
  lerp_interpolation = asanyarray(add(a, diff_b_a * t, out=out))


### Load Hyper & Meta-Log

In [4]:
# score_dict = {
#     "OpenAI-ES": np.array([[0], [1], [2], [3]]),
#     "PGPE": np.array([[0], [1], [2], [3]]) + 1,
#     "ARS": np.array([[0], [1], [2], [3]]) + 2,
#     "SNES": np.array([[0], [1], [2], [3]]) + 3,
#     "Sep-CMA-ES": np.array([[0], [1], [2], [3]]) + 4,
#     "LES": np.array([[0], [1], [2], [3]]) + 5,
#     "Gaussian GA": np.array([[0], [1], [2], [3]]) + 6,
#     "SAMR-GA": np.array([[0], [1], [2], [3]]) + 7,
#     "GESMR-GA": np.array([[0], [1], [2], [3]]) + 8,
#     "LGA": np.array([[0], [1], [2], [3]]) + 9,
# }


In [5]:
plot_rliable(score_dict, "iqm", reps=100)

(<Figure size 3600x6000 with 1 Axes>,
 <Axes: title={'center': 'Interquartile Mean (IQM)'}, xlabel='OpenES Norm. Performance'>)

# Per Seed Results Loading

In [6]:
def score_normalization(scores, min_score, max_score):
  norm_score = (scores - min_score)/(max_score - min_score)
  return norm_score


def load_and_normalize(log_path):
    meta_log, hyper_log = load_result_logs(log_path, aggregate_seeds=True)
    seeds = hyper_log.seeds.iloc[0]
    openai_id = hyper_log[hyper_log.strategy_name == "OpenES"].run_id.iloc[0]
    min_perf = meta_log[openai_id].stats.test_eval_perf.mean[0]
    max_perf = meta_log[openai_id].stats.test_eval_perf.mean[-1]
    meta_log, hyper_log = load_result_logs(log_path, aggregate_seeds=False)

    results_dict = {}
    for i, s in enumerate(hyper_log.strategy_name):
        all_scores = []
        for sid in seeds:
          str_id = hyper_log[hyper_log.strategy_name == s].run_id.iloc[0]
          perf = meta_log[str_id]["seed_" + str(sid)].stats.test_eval_perf[-1]
          all_scores.append(perf)
        results_dict[s] = score_normalization(np.array(all_scores), min_perf, max_perf)

    return results_dict

In [7]:
load_and_normalize("../experiments/fmnist_classify/eval/")

{'OpenES': array([1.00034  , 1.0038697, 0.9994249, 1.0028237, 0.993542 ],
       dtype=float32),
 'PGPE': array([0.99746394, 0.9901431 , 0.99981695, 0.9944571 , 0.99458784],
       dtype=float32),
 'ARS': array([0.9735404 , 0.97785455, 0.9722332 , 0.96909565, 0.9756321 ],
       dtype=float32),
 'SNES': array([0.9868748 , 0.98582894, 0.9856982 , 0.97916174, 0.9839988 ],
       dtype=float32),
 'Sep_CMA_ES': array([0.97667795, 0.9740634 , 0.9744555 , 0.98033834, 0.96935713],
       dtype=float32),
 'LES': array([0.9518393, 0.9430806, 0.9498784, 0.9511857, 0.9458259],
       dtype=float32),
 'SimpleGA': array([0.75509197, 0.74319565, 0.6846289 , 0.70933676, 0.7409732 ],
       dtype=float32),
 'SAMR_GA': array([0.9400738, 0.939943 , 0.9421655, 0.937067 , 0.939943 ],
       dtype=float32),
 'GESMR_GA': array([0.9203337, 0.916281 , 0.9113134, 0.9170654, 0.8764087],
       dtype=float32),
 'LGA': array([0.82111025, 0.8162732 , 0.7933956 , 0.80895245, 0.8205872 ],
       dtype=float32)}

In [8]:
all_dicts = []
for dir in result_dirs:
    d = load_and_normalize(dir)
    all_dicts.append(d)

dd = defaultdict(list)

for d in all_dicts:
    for key, value in d.items():
        dd[key].append(value)

score_dict = dict(dd)

for k, v in score_dict.items():
    score_dict[k] = np.stack(v)

del score_dict["OpenES"]

In [9]:
score_dict

{'PGPE': array([[ 9.9746394e-01,  9.9014312e-01,  9.9981695e-01,  9.9445713e-01,
          9.9458784e-01],
        [ 1.0255384e+00,  1.0260195e+00,  1.0188516e+00,  1.0168550e+00,
          1.0140221e+00],
        [ 9.2313808e-01,  9.1491693e-01,  9.2098087e-01,  9.6841753e-01,
          1.0224729e+00],
        [ 6.9603539e-01,  6.5493035e-01,  6.8358934e-01,  9.3821859e-01,
          7.5746274e-01],
        [ 9.8228198e-01,  9.9525481e-01,  9.8788363e-01,  1.0000653e+00,
          9.9194801e-01],
        [ 9.9659932e-01,  8.2681966e-01,  1.1613034e+00,  1.0874531e+00,
          9.3949860e-01],
        [ 2.3502278e+00,  6.8337172e-03, -1.7084238e-03,  1.9191344e-01,
          1.8906605e-01],
        [-3.9904381e-04, -2.3942553e-03, -2.3942553e-03, -2.3942553e-03,
          1.5961677e-03],
        [ 9.7917330e-01,  1.0189042e+00,  9.5161808e-01,  9.5610380e-01,
          1.0768985e+00]], dtype=float32),
 'ARS': array([[ 9.7354043e-01,  9.7785455e-01,  9.7223318e-01,  9.6909565e-01,
    

In [10]:
score_dict['Sep-CMA-ES'] = score_dict['Sep_CMA_ES'] 
score_dict['SAMR-GA'] = score_dict['SAMR_GA'] 
score_dict['GESMR-GA'] = score_dict['GESMR_GA'] 
del score_dict["Sep_CMA_ES"]
del score_dict["SAMR_GA"]
del score_dict["GESMR_GA"]

In [11]:
plot_rliable(score_dict, "iqm", reps=100)

(<Figure size 3600x6000 with 1 Axes>,
 <Axes: title={'center': 'Interquartile Mean (IQM)'}, xlabel='OpenES Norm. Performance'>)